In [37]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow import keras
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

In [38]:
DATA_DIR = "data_all"
IMG_SIZE = 128

fruit_classes = ["ananas", "banane", "tomate", "papaye", "non_fruit"]
maturity_classes = ["pas_mur", "mur", "trop_mur"]

X = []
y_fruit = []
y_maturity = []


for fruit in fruit_classes:
    fruit_path = os.path.join(DATA_DIR, fruit)
    if not os.path.isdir(fruit_path):
        continue

    if fruit == "non_fruit":
        files = [f for f in os.listdir(fruit_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        for filename in files:
            img_path = os.path.join(fruit_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            X.append(img)
            y_fruit.append(fruit)
            y_maturity.append(3)
        continue

    for subdir in os.listdir(fruit_path):
        subdir_path = os.path.join(fruit_path, subdir)
        if not os.path.isdir(subdir_path):
            continue

        files = [f for f in os.listdir(subdir_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        for filename in files:
            img_path = os.path.join(subdir_path, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            X.append(img)
            y_fruit.append(fruit)

            if subdir in maturity_classes:
                y_maturity.append(maturity_classes.index(subdir))
            else:
                y_maturity.append(3)


X = np.array(X, dtype=np.float32)
fruit_indices = [fruit_classes.index(f) for f in y_fruit]
y_fruit = to_categorical(fruit_indices, num_classes=len(fruit_classes))

y_maturity = to_categorical(y_maturity, num_classes=len(maturity_classes)+1)
maturity_mask = (np.argmax(y_maturity, axis=1) != 3).astype("float32")

X_train, X_test, y_fruit_train, y_fruit_test, y_maturity_train, y_maturity_test, mask_train, mask_test = train_test_split(
    X, y_fruit, y_maturity, maturity_mask, test_size=0.2, random_state=42
)

print("Train :", X_train.shape, y_fruit_train.shape, y_maturity_train.shape)
print("Test  :", X_test.shape, y_fruit_test.shape, y_maturity_test.shape)


Train : (3816, 128, 128, 3) (3816, 5) (3816, 4)
Test  : (955, 128, 128, 3) (955, 5) (955, 4)


In [39]:
IMG_SIZE = 128
num_fruit_classes = 5
num_maturity_classes = 3


input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

fruit_output = Dense(num_fruit_classes, activation='softmax', name='fruit_output')(x)

maturity_output = Dense(num_maturity_classes + 1, activation='softmax', name='maturity_output')(x)

model = Model(inputs=input_layer, outputs=[fruit_output, maturity_output])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'fruit_output': 'categorical_crossentropy',
        'maturity_output': 'categorical_crossentropy'
    },
    metrics={
        'fruit_output': 'accuracy',
        'maturity_output': 'accuracy'
    }
)

model.summary()

history = model.fit(
    X_train,
    {
        'fruit_output': y_fruit_train,
        'maturity_output': y_maturity_train
    },
    validation_data=(
        X_test,
        {
            'fruit_output': y_fruit_test,
            'maturity_output': y_maturity_test
        }
    ),
    epochs=15,
    batch_size=32
)

model.save("fruit_maturity_detector.keras")

print("Modèle sauvegardé dans fruit_maturity_detector.keras")


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 128, 128,  │        896 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 64, 64,    │          0 │ conv2d_12[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 64, 64,    │     18,496 │ max_pooling2d_12… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 32, 32,    │          0 │ conv2d_13[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 32, 32,    │     73,856 │ max_pooling2d_13… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 16, 16,    │          0 │ conv2d_14[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 32768)     │          0 │ max_pooling2d_14… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │  8,388,864 │ flatten_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fruit_output        │ (None, 5)         │      1,285 │ dropout_4[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ maturity_output     │ (None, 4)         │      1,028 │ dropout_4[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,484,425 (32.37 MB)

 Trainable params: 8,484,425 (32.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 34s 267ms/step - fruit_output_accuracy: 0.9541 - fruit_output_loss: 0.1586 - loss: 0.7099 - maturity_output_accuracy: 0.7822 - maturity_output_loss: 0.5519 - val_fruit_output_accuracy: 0.9906 - val_fruit_output_loss: 0.0385 - val_loss: 0.3906 - val_maturity_output_accuracy: 0.8901 - val_maturity_output_loss: 0.3513
Epoch 2/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 37s 304ms/step - fruit_output_accuracy: 0.9893 - fruit_output_loss: 0.0399 - loss: 0.3585 - maturity_output_accuracy: 0.8852 - maturity_output_loss: 0.3193 - val_fruit_output_accuracy: 0.9969 - val_fruit_output_loss: 0.0232 - val_loss: 0.2863 - val_maturity_output_accuracy: 0.8995 - val_maturity_output_loss: 0.2633
Epoch 3/15
120/120 ━━━━━━━━━━━━━━━━━━━━ 37s 308ms/step - fruit_output_accuracy: 0.9950 - fruit_output_loss: 0.0214 - loss: 0.2720 - maturity_output_accuracy: 0.9104 - maturity_output_loss: 0.2532 - val_fruit_output_accuracy: 0.9979 - val_fruit_output_loss: 0.0126 - val_loss: 0.2168 - v

In [40]:
results = model.evaluate(
    X_test,
    {
        'fruit_output': y_fruit_test,
        'maturity_output': y_maturity_test
    },
    batch_size=32
)

print("Résultats de l'évaluation :", results)


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - fruit_output_accuracy: 0.9990 - fruit_output_loss: 0.0020 - loss: 0.2518 - maturity_output_accuracy: 0.9225 - maturity_output_loss: 0.2503
Résultats de l'évaluation : [0.2518260180950165, 0.001980318920686841, 0.2502804398536682, 0.9989528656005859, 0.9225130677223206]


In [41]:
y_fruit_test_labels = np.argmax(y_fruit_test, axis=1)
y_fruit_pred_labels = np.argmax(model.predict(X_test)[0], axis=1)

print(confusion_matrix(y_fruit_test_labels, y_fruit_pred_labels))
print(classification_report(y_fruit_test_labels, y_fruit_pred_labels, target_names=fruit_classes))


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step
[[ 17   0   0   0   0]
 [  0 496   1   0   0]
 [  0   0 401   0   0]
 [  0   0   0  38   0]
 [  0   0   0   0   2]]
              precision    recall  f1-score   support

      ananas       1.00      1.00      1.00        17
      banane       1.00      1.00      1.00       497
      tomate       1.00      1.00      1.00       401
      papaye       1.00      1.00      1.00        38
   non_fruit       1.00      1.00      1.00         2

    accuracy                           1.00       955
   macro avg       1.00      1.00      1.00       955
weighted avg       1.00      1.00      1.00       955



TEST 1

In [44]:
img = cv2.imread("banane_mur.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (128, 128))
img = img / 255.0
img = np.expand_dims(img, axis=0)

pred_fruit, pred_maturity = model.predict(img)

fruit_idx = np.argmax(pred_fruit)
maturity_idx = np.argmax(pred_maturity)

print("Fruit prédit :", fruit_classes[fruit_idx])
print("Maturité prédite :", maturity_classes[maturity_idx])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Fruit prédit : banane
Maturité prédite : pas_mur


TEST 2

In [50]:

model_path = "fruit_maturity_detector.keras"
IMG_SIZE = 128

fruit_classes = ["ananas", "banane", "tomate", "papaye", "non_fruit"]
maturity_classes = ["pas_mur", "mur", "trop_mur"]

model = load_model(model_path)

def predict_image(image_path):
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : impossible de lire l'image {image_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred_fruit, pred_maturity = model.predict(img)
    fruit_idx = np.argmax(pred_fruit)
    maturity_idx = np.argmax(pred_maturity)

    if fruit_classes[fruit_idx] == "non_fruit":
        print(f"Image : {image_path} → Ce n'est pas un fruit.")
    else:
        print(f"Image : {image_path} → Fruit : {fruit_classes[fruit_idx]}, Maturité : {maturity_classes[maturity_idx]}")

predict_image("tomate_mur.jpg")
predict_image("banane_mur.jpg")
predict_image("banane_mur2.png")
predict_image("non_fruit3.jpg")
predict_image("non_fruit1.jpg")
predict_image("7_dos.jpg")

predict_image("tomate_mur1.png")
predict_image("papaye_mur.jpg")
predict_image("banane_pas_mur1.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Image : tomate_mur.jpg → Fruit : tomate, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image : banane_mur.jpg → Fruit : banane, Maturité : pas_mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image : banane_mur2.png → Fruit : banane, Maturité : trop_mur
Erreur : impossible de lire l'image non_fruit3.jpg
Erreur : impossible de lire l'image non_fruit1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image : 7_dos.jpg → Fruit : ananas, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image : tomate_mur1.png → Fruit : tomate, Maturité : trop_mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image : papaye_mur.jpg → Fruit : papaye, Maturité : mur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Image : banane_pas_mur1.png → Fruit : tomate, Maturité : pas_mur
